In [ ]:
import numpy as np
import matplotlib.pylab as plt

%reload_ext autoreload
%autoreload 2

In [ ]:
FS = 32000
N_BUFFER = 2048
N_FREQS = 32

f_noises = [300, 450]

f = FS/N_BUFFER*(32.5)
print(f)
times = np.arange(0, 1, 1/FS)
signal_t = 1.0 * np.sin(2*np.pi*f*times)
for f_noise in f_noises:
    signal_t += 0.8*np.sin(2*np.pi*f_noise*times)

buffer_t = signal_t[:N_BUFFER]
plt.scatter(times[:N_BUFFER], buffer_t, s=1.0)
plt.xlim(0, 1/f)

In [ ]:
buffer_f = 2 / N_BUFFER * np.fft.rfft(buffer_t)
freq = np.fft.rfftfreq(N_BUFFER, 1/FS)
plt.scatter(freq, np.abs(buffer_f))
plt.xlim(0, f + 200)

In [ ]:
f_bin = int(f * N_BUFFER / FS) + 1
print(f)
print(freq[f_bin])
bins = np.arange(f_bin - N_FREQS / 2, f_bin + N_FREQS / 2).astype(np.int)

plt.scatter(freq[bins], np.abs(buffer_f[bins]))

In [ ]:
buffer_f_padded = np.zeros(N_BUFFER // 2 + 1, dtype=np.complex)
buffer_f_padded[bins] = buffer_f[bins]

In [ ]:
buffer_t_padded = np.fft.irfft(buffer_f_padded, n=N_BUFFER)

In [ ]:
plt.scatter(times[:N_BUFFER], buffer_t_padded)
plt.xlim(0, 2/f)

In [ ]:
pad_factor = 10
buffer_f_cont = np.fft.rfft(buffer_t_padded, n=pad_factor*N_BUFFER)
freq_cont = np.fft.rfftfreq(pad_factor*N_BUFFER, d=1/FS)

In [ ]:
plt.plot(freq_cont, np.abs(buffer_f_cont))
plt.scatter(freq, np.abs(buffer_f))
plt.xlim(0, f + 200)

In [ ]:
from scipy.interpolate import lagrange
from frequency_analysis import fit_peak

fig, ax = plt.subplots()

n = 4
ax.scatter(freq[bins], np.abs(buffer_f[bins]), color='C0')
chosen_bins = range(f_bin-n,f_bin+n) 
x = freq[chosen_bins]
y = np.abs(buffer_f)[chosen_bins]
plt.scatter(x, y, color='C1')

poly = lagrange(x, y)
freq_lagrange = np.linspace(min(x), max(x), 100)
y_lagrange = np.abs(poly(freq_lagrange))
ax.plot(freq_lagrange, y_lagrange, color='C2')
print(max(y_lagrange))

mag, p = fit_peak(np.abs(buffer_f), f_bin)
freq_p = (f_bin + p)*FS/N_BUFFER
ax.plot([f0, f0], [0, mag], c='C3')
print(mag, freq_p)

In [ ]:
from frequency_analysis import interpolate_peak
    
fig, ax = plt.subplots()
mag, f0 = interpolate_peak(buffer_f_padded, freq, ax=ax)
ax.plot([f0, f0], [0, mag], c='C1')
ax.set_xlim(0, 1000)